# Apache Zookeeper

Координационный сервис для распределенных приложений. Логически представляет из себя двевовидную структуру, наподобие файловой системы, к каждому узлу которой можно присвоить значение. 

Запускаем его в `Docker` вместе с `Kafka`:
```bash
docker run --rm -p 2181:2181 zookeeper:3.7.0
```

### Подключение

In [3]:
from kazoo.client import KazooClient

zk = KazooClient(hosts='127.0.0.1:2181')
zk.start()

### Создание элемента

In [4]:
zk.ensure_path('/node/a')
zk.set('/node/a', b'hello')

ZnodeStat(czxid=3, mzxid=4, ctime=1714081276621, mtime=1714081276631, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=5, numChildren=0, pzxid=3)

In [5]:
zk.ensure_path('/database2/host')
zk.set('/database2/host', b'192.168.1.2')

ZnodeStat(czxid=6, mzxid=7, ctime=1714081278940, mtime=1714081278951, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=11, numChildren=0, pzxid=6)

In [6]:
zk.get('/database2/host')

(b'192.168.1.2',
 ZnodeStat(czxid=6, mzxid=7, ctime=1714081278940, mtime=1714081278951, version=1, cversion=0, aversion=0, ephemeralOwner=0, dataLength=11, numChildren=0, pzxid=6))

можно иначе

In [7]:
zk.create('/node/b', b'hello2')

'/node/b'

### Отслеживание изменений

In [8]:
def callback(p):    
    print('Event detected: ', p)
    
zk.get('/node/a', callback)
zk.set('/node/a', b'hello2')

Event detected:  WatchedEvent(type='CHANGED', state='CONNECTED', path='/node/a')


ZnodeStat(czxid=3, mzxid=10, ctime=1714081276621, mtime=1714081285987, version=2, cversion=0, aversion=0, ephemeralOwner=0, dataLength=6, numChildren=0, pzxid=3)

### Транзакции

In [9]:
with zk.transaction() as t:
    t.create('/node/c', b'c value')
    t.create('/node/d', b'd value')
zk.get('/node/c')    

(b'c value',
 ZnodeStat(czxid=11, mzxid=11, ctime=1714081288200, mtime=1714081288200, version=0, cversion=0, aversion=0, ephemeralOwner=0, dataLength=7, numChildren=0, pzxid=11))

### Блокировки

In [10]:
import threading
import time

def thread_func(num):
    with zk.Lock('/node/a') as lock:
        print('Thread #{} lock'.format(num))
        zk.ensure_path('/node/a/c')
        zk.set('/node/a/c', b'hello')
        time.sleep(2)
        print('Thread #{} unlock'.format(num))

for i in range(3):
    threading.Thread(target=thread_func, args=(i,)).start()

Thread #0 lock
Thread #0 unlock
Thread #1 lock
Thread #1 unlock
Thread #2 lock
Thread #2 unlock


### Election

In [11]:
def thread_func(num):
    election = zk.Election("/node")
    
    def election_func():
        print('Election won: {}'.format(num))
    
    election.run(election_func)


for i in range(3):
    threading.Thread(target=thread_func, args=(i,)).start()

Election won: 0
Election won: 1
Election won: 2


### Очереди

In [12]:
if zk.exists('/queue'):
    zk.delete('/queue', recursive=True)
    
q = zk.Queue('/queue')
q.put(b'1', 10)
q.put(b'2', 0)

In [13]:
q.get()

b'2'

### Счетчики

In [14]:
if zk.exists('/counter'):
    zk.delete('/counter', recursive=True)
    
counter = zk.Counter('/counter')
counter += 20
counter -= 5
counter.value

15

In [15]:
counter += 100

In [ ]:
zk.get('/counter')

(b'115',
 ZnodeStat(czxid=32, mzxid=35, ctime=1714081303031, mtime=1714081305539, version=3, cversion=0, aversion=0, ephemeralOwner=0, dataLength=3, numChildren=0, pzxid=32))

Connection dropped: socket connection broken
Connection dropped: socket connection broken
Transition to CONNECTING
Transition to CONNECTING
Connection dropped: socket connection error: Connection reset by peer
Connection dropped: socket connection error: Connection reset by peer
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropped: socket connection error: Connection refused
Connection dropp